In [1]:
#参考サイト
#http://ushinji.hatenablog.com/entry/2017/11/23/092614  ランダムフォレストによる毒キノコの分類
#https://www.kaggle.com/uciml/mushroom-classification  カラムの意味
#https://www.haya-programming.com/entry/2018/03/14/112454　精度の評価 
#https://blog.amedama.jp/entry/2017/12/18/005311　制度の評価
#http://ibisforest.org/index.php?F%E5%80%A4　F値（評価尺度）の意味（レジュメ）
#https://pythondatascience.plavox.info/scikit-learn/%E5%88%86%E9%A1%9E%E7%B5%90%E6%9E%9C%E3%81%AE%E3%83%A2%E3%83%87%E3%83%AB%E8%A9%95%E4%BE%A1
#↑tp,tn,fp,fnとかの数の計算方法
#https://qiita.com/fukuit/items/b94e58191790bfce7f8b　　tpとかの行列での表示

In [2]:
#必要なモジュールをインポート

In [3]:
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn import datasets

In [4]:
#csvファイルの読み込み
#キノコのデータセット

In [5]:
mush = pd.read_csv('mushrooms.csv')
pd.set_option("display.max_columns", 23)
pd.set_option('display.max_rows', 600)
mush

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,e,e,s,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,e,c,s,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,e,e,s,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,t,e,s,s,w,w,p,w,o,e,n,a,g
5,e,x,y,y,t,a,f,c,b,n,e,c,s,s,w,w,p,w,o,p,k,n,g
6,e,b,s,w,t,a,f,c,b,g,e,c,s,s,w,w,p,w,o,p,k,n,m
7,e,b,y,w,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,s,m
8,p,x,y,w,t,p,f,c,n,p,e,e,s,s,w,w,p,w,o,p,k,v,g
9,e,b,s,y,t,a,f,c,b,g,e,c,s,s,w,w,p,w,o,p,k,s,m


In [6]:
#カラムのタイプを見てみる

In [7]:
mush.dtypes

class                       object
cap-shape                   object
cap-surface                 object
cap-color                   object
bruises                     object
odor                        object
gill-attachment             object
gill-spacing                object
gill-size                   object
gill-color                  object
stalk-shape                 object
stalk-root                  object
stalk-surface-above-ring    object
stalk-surface-below-ring    object
stalk-color-above-ring      object
stalk-color-below-ring      object
veil-type                   object
veil-color                  object
ring-number                 object
ring-type                   object
spore-print-color           object
population                  object
habitat                     object
dtype: object

In [8]:
#欠損値の数を調べる

In [9]:
mush.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
class                       8124 non-null object
cap-shape                   8124 non-null object
cap-surface                 8124 non-null object
cap-color                   8124 non-null object
bruises                     8124 non-null object
odor                        8124 non-null object
gill-attachment             8124 non-null object
gill-spacing                8124 non-null object
gill-size                   8124 non-null object
gill-color                  8124 non-null object
stalk-shape                 8124 non-null object
stalk-root                  8124 non-null object
stalk-surface-above-ring    8124 non-null object
stalk-surface-below-ring    8124 non-null object
stalk-color-above-ring      8124 non-null object
stalk-color-below-ring      8124 non-null object
veil-type                   8124 non-null object
veil-color                  8124 non-null object
ring-number

In [10]:
#↑欠損値はひとつもない

In [11]:
#ラベルエンコーダーで、文字を数字にする

In [12]:
from sklearn import preprocessing

In [13]:
for column in mush.columns:
    le = preprocessing.LabelEncoder()
    le.fit(mush[column])
    label_encoded_column = le.transform(mush[column])
    mush[column] = le.transform(mush[column])
mush

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,0,3,2,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,0,2,2,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,0,2,2,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,0,3,2,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,1,3,2,2,7,7,0,2,1,0,3,0,1
5,0,5,3,9,1,0,1,0,0,5,0,2,2,2,7,7,0,2,1,4,2,2,1
6,0,0,2,8,1,0,1,0,0,2,0,2,2,2,7,7,0,2,1,4,2,2,3
7,0,0,3,8,1,3,1,0,0,5,0,2,2,2,7,7,0,2,1,4,3,3,3
8,1,5,3,8,1,6,1,0,1,7,0,3,2,2,7,7,0,2,1,4,2,4,1
9,0,0,2,9,1,0,1,0,0,2,0,2,2,2,7,7,0,2,1,4,2,3,3


In [14]:
#目的変数と説明変数に分ける

In [15]:
X = mush[['bruises','odor','veil-type','cap-color']]
type(X)

pandas.core.frame.DataFrame

In [16]:
y = mush['class']
type(y)

pandas.core.series.Series

In [17]:
#trainデータとtestデータに分ける

In [18]:
from sklearn.model_selection import train_test_split
y_train,y_test,X_train,X_test = train_test_split(y,X,test_size=0.4)

In [19]:
#ランダムフォレストを定義し、trainデータでfittingする

In [20]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [21]:
#fitされた学習器にtestデータを当てはめ、予測を行う

In [22]:
y_pred = clf.predict(X_test)

In [23]:
#精度を計算し、評価する

In [24]:
#識別結果の正解、不正解を考える
#（予測の値、実際の値）
#（正、正）→　真陽性　TP:True Positive
#（正、負）→　偽陽性　FP:False Positive
#（負、正）→　偽陰性　FN:False Negative
#（負、負）→　真陰性　TN:True Negative

In [25]:
#これら４つの数を調べる

In [47]:
from sklearn.metrics import confusion_matrix
tn,fp,fn,tp = confusion_matrix(y_test,y_pred).ravel()
(tp,fn,fp,tn)

(1560, 26, 0, 1664)

In [48]:
from sklearn.metrics import confusion_matrix
print (confusion_matrix(y_test, y_pred))

[[1664    0]
 [  26 1560]]


In [49]:
#☆正解率　Accuracy

In [50]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

0.992


In [51]:
#☆正解率を手計算で

In [52]:
accuracy = (tp+tn)/(tp+fn+fp+tn)
print(accuracy)

0.992


In [53]:
#☆精度　Precision

In [54]:
from sklearn.metrics import precision_score
print(precision_score(y_test,y_pred))

1.0


In [55]:
#☆適合率を手計算で

In [56]:
precision = tp/(tp+fp)
print(precision)

1.0


In [57]:
#☆再現率　Recall

In [58]:
from sklearn.metrics import recall_score
print(recall_score(y_test,y_pred))

0.9836065573770492


In [59]:
#☆再現率を手計算で

In [60]:
recall = tp/(tp+fn)
print(recall)

0.9836065573770492


In [61]:
#☆F1値

In [62]:
from sklearn.metrics import f1_score
print(f1_score(y_test,y_pred))

0.9917355371900827


In [63]:
#☆F1値を手計算で

In [64]:
f1 = (2*recall*precision)/(recall+precision)
print(f1)

0.9917355371900827
